In [3]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from threading import Thread


In [12]:
NUM_DRIVER = os.cpu_count()
options = webdriver.ChromeOptions()
options.headless = True

In [13]:
class Reviews(Thread):
    def __init__(self, driver_id, product_list, index_start=0, index_end= 314):
        Thread.__init__(self,)
        self.driver_id = driver_id
        self.product_list = product_list
        self.index_start = index_start
        self.index_end = index_end
        
        self.review_df = pd.DataFrame(columns=['product', 'username', 'user_rating',
                                               'title', 'user_review', 'review_posted_time'])
        self.idx = 0

        print('Initializing driver', self.driver_id)
        self.driver = webdriver.Chrome(options=options)

    def run(self) -> None:
        for index in range(self.index_start, self.index_end + 1):
            if index % NUM_DRIVER == self.driver_id:

                product_name = self.product_list['Name'][index]
                product_link = self.product_list['Link'][index]
                pages = self.product_list['Pages'][index]
                for page in range(pages):
                    # print('\nIndex - Driver : Page / Pages  :  Product Name')
                    print(index, '-', self.driver_id, ':', page + 1, '/', pages, ' : ', product_name)

                    # print('Page:', i, '    ', 'Driver:', driver_id)
                    replace_string = 'product-reviews'
                    review_link = product_link[0: product_link.index(
                        'FLIPKART') + 8]
                    review_link = review_link.split('/')
                    review_link[4] = replace_string
                    review_link = '/'.join(review_link)
                    # page = zipped[2]

                    url = review_link + '&page=' + str(page+1)
                    self.driver.get(url)
                    try:
                        WebDriverWait(self.driver, 2).until(
                            EC.presence_of_element_located(
                                (By.XPATH, "//span[text()='READ MORE']")))

                        element = self.driver.find_elements(
                            By.XPATH, "//span[text()='READ MORE']")  # For expanding reviews

                        for i in element:
                            if i.text == 'READ MORE':
                                i.click()
                    except:
                        pass
                    
                    html = self.driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    reviews = soup.find_all(
                        'div', attrs={'class': 'col _2wzgFH K0kLPL'})
                    for review in reviews:
                        row = review.find(
                            'div', attrs={'class': 'row'})
                        user_rating = row.find('div').text
                        title = row.find('p').text
                        user_review = review.find(
                            'div', attrs={'class': 't-ZTKy'}).text.replace('READ MORE', '')

                        res = [product_name, user_rating,
                            title, user_review]
                        self.review_df.loc[self.idx] = res
                        self.idx += 1

In [7]:
product_list = pd.read_csv('unique_product_list_with_ratings.csv')
q = product_list['Pages'].quantile(.95)
product_list = product_list[product_list['Pages'] < q]

product_list = product_list.sort_values(by='Pages').reset_index(drop=True)
# product_list = product_list.reset_index(drop=True)

product_list['Pages'] = product_list['Pages'].astype(dtype='int')
product_list['Total Reviews'] = product_list['Total Reviews'].apply(lambda x: x.replace(',', '')).astype(
    dtype='int')
print(len(product_list))
product_list.head(1)


317


,Brand,Name,Price,Link,Features,Overall Rating,Total Reviews,Pages,5,4,3,2,1
0,Asus Mobile,ASUS ZenFone 5Z,"36,299",https://www.flipkart.com/asus-zenfone-5z-midni...,['6 GB RAM | 128 GB ROM | Expandable Upto 2 TB...,4.5,4616,462,15005,4566,979,339,1052


Fetching data in chuncks

In [ ]:
idx = [150, 158]

for j in range(len(idx) - 1):
    INDEX_START = idx[j] + 1
    INDEX_END = idx[j + 1]

    # product_list['Total Reviews'].iloc[INDEX_START: INDEX_END + 1].sum()
    threads = []
    start = time.time()
    for i in range(NUM_DRIVER):
        thread = Reviews(driver_id=i, product_list=product_list, index_start=INDEX_START, index_end=INDEX_END)
        threads.append(thread)

    print('\nIndex - Driver : Page / Pages  :  Product Name')
    for i in range(NUM_DRIVER):
        threads[i].start()
        
    for i in range(NUM_DRIVER):
        threads[i].join()

    time.sleep(1)

    review_df = pd.concat([pd.DataFrame(data=threads[i].review_df).T for i in range(NUM_DRIVER)], ignore_index=True)
    
    time.sleep(1)
    print('Total  vs  Fetched')
    total = product_list['Total Reviews'].iloc[INDEX_START: INDEX_END + 1].sum()
    fetched = len(review_df)
    print(total, ' vs ', fetched)
    
    review_df.to_csv(f'reviews_{INDEX_START}_to_{INDEX_END}.csv', index=False)
    


Printing number of reviews for each chunk

In [8]:
idx = [-1, 25, 50, 75, 100]
print('Total Reviews:', product_list['Total Reviews'].sum())
for i in range(len(idx) - 1):
    INDEX_START = idx[i] + 1
    INDEX_END = idx[i + 1]
    reviews = product_list['Total Reviews'].iloc[INDEX_START: INDEX_END + 1].sum()
    print(f'Total Reviews from {INDEX_START} to {INDEX_END}:', reviews)


Total Reviews: 672580
Total Reviews from 0 to 25: 100641
Total Reviews from 26 to 50: 98951
Total Reviews from 51 to 75: 61557
Total Reviews from 76 to 100: 72715


Printing lenght of fetched data (from files)

In [6]:
reviews = 0
for i in range(len(idx) - 1):
    INDEX_START = idx[i] + 1
    INDEX_END = idx[i + 1]

    file = pd.read_csv(f'reviews/reviews_{INDEX_START}_to_{INDEX_END}.csv')
    reviews += len(file)
    print(f'reviews_{INDEX_START}_to_{INDEX_END}:', len(file))
print(reviews)

FileNotFoundError: [Errno 2] No such file or directory: 'reviews/reviews_151_to_160.csv'

Checking for uniform data for each driver

In [10]:
ps = [0 for i in range(NUM_DRIVER)]

for i in range(len(product_list)):
    ps[i % NUM_DRIVER] += product_list['Pages'][i]

print(ps)

[8652, 8764, 9108, 7395, 7524, 7792, 7968, 8343]


Fetching the Overall Ratings and Reviews

In [ ]:
product_list = pd.read_csv('unique_product_list.csv')
product_list = pd.concat([product_list, pd.DataFrame(
    columns=['Overall Rating','Total Reviews', 'Pages', '5', '4', '3', '2', '1'])], axis=1)
idx_product = 0

driver = webdriver.Chrome(options=options)
for index in range(len(product_list)):
    # try:
    if True:
        product_name = product_list['Name'].iloc[index]
        product_link = product_list['Link'].iloc[index]

        replace_string = 'product-reviews'
        review_link = product_link[0: product_link.index('FLIPKART') + 8]
        review_link = review_link.split('/')
        review_link[4] = replace_string
        review_link = '/'.join(review_link)

        driver.get(review_link)
        
        print()
        print(index, '- Product:', product_name)
        print('Link:', review_link)
        pages = 1
        try:
            pages = driver.find_element(
                By.XPATH, "//*[@class='_2MImiq _1Qnn1K']").text
            pages = int(pages[10:pages.index('\n')].replace(',', ''))  # String
            print('Pages:', pages)
        except:
            print('Sigle page review')
        
        soup = BeautifulSoup(driver.page_source, 'lxml')

        overall_rating = soup.find(class_='_2d4LTz')
        overall_rating = float(overall_rating.text)  # Float
        
        total_reviews = soup.find_all('div', attrs={'class': 'row _2afbiS'})[1].text
        total_reviews = total_reviews[ : total_reviews.index('Reviews') - 1]

        review_ratings, rating_count = soup.find_all(
            class_='_26f_zl'), soup.find_all(class_='_1uJVNT')  # List
        for i in range(len(rating_count)):
            review_ratings[i] = int(review_ratings[i].text)
            rating_count[i] = int(rating_count[i].text.replace(',', ''))

        ratings_star_counts = dict(
            zip(review_ratings, rating_count))  # dictionary

        product_list['Overall Rating'].iloc[idx_product] = overall_rating
        product_list['Total Reviews'].iloc[idx_product] = total_reviews
        product_list['Pages'].iloc[idx_product] = pages
        product_list['5'].iloc[idx_product] = ratings_star_counts[5]
        product_list['4'].iloc[idx_product] = ratings_star_counts[4]
        product_list['3'].iloc[idx_product] = ratings_star_counts[3]
        product_list['2'].iloc[idx_product] = ratings_star_counts[2]
        product_list['1'].iloc[idx_product] = ratings_star_counts[1]
        idx_product += 1

product_list.to_csv('unique_product_list_with_ratings.csv', index=False)
